# Prática spark
## Objetivo
El objetivo de esta práctica es realizar el procesado de un fichero de logs para poder
## Práctica
### Conexión
El primer paso para realizar la práctica es establecer la conexión con el cluster de spark

In [1]:
from pyspark.sql import SparkSession
from  pyspark.sql.functions import col, concat_ws,split, explode ,desc, to_timestamp,from_unixtime,unix_timestamp, translate, create_map, lit,udf,StringType,map_keys
from datetime import datetime
from itertools import chain

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

22/11/21 17:14:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Lectura
Una vez que se ha establecido la conexión con el cluster de spark, es necesario cargar el archivo de texto que se va a tratar.
Para esta práctica se va a leer como un dataframe

In [2]:
ficheroDataframe = spark.read.text("logs.txt")
ficheroDataFrameRenamed = ficheroDataframe.withColumnRenamed("value","datos")
# Se muestran los primeros 20 elementos
ficheroDataFrameRenamed.show()

+--------------------+
|               datos|
+--------------------+
|01-11-2022 13:24:...|
|01-11-2022 13:24:...|
|01-11-2022 13:24:...|
|01-11-2022 13:24:...|
|01-11-2022 13:24:...|
|01-11-2022 13:24:...|
|01-11-2022 13:24:...|
|01-11-2022 13:24:...|
|01-11-2022 13:24:...|
|01-11-2022 13:24:...|
|01-11-2022 13:24:...|
|01-11-2022 13:24:...|
|01-11-2022 13:24:...|
|01-11-2022 13:24:...|
|01-11-2022 13:24:...|
|01-11-2022 13:24:...|
|01-11-2022 13:24:...|
|01-11-2022 13:24:...|
|01-11-2022 13:24:...|
|01-11-2022 13:24:...|
+--------------------+
only showing top 20 rows



### Preprocesado

Una vez que se ha cargado el fichero, será necesario dividir la columna por el carácter "-"

In [3]:
ficheroDataFrameRenamed.filter(col("datos").isNull()).show()

+-----+
|datos|
+-----+
+-----+



Además comprobamos que todas las filas tienen el carácter por el que vamos a separar el string

In [4]:
ficheroDataFrameRenamed.filter(~ col("Datos").contains(" - ")).show()

+-----+
|datos|
+-----+
+-----+



Se eliminan los valores Nan

In [5]:
ficheroDataFrameRenamed = ficheroDataFrameRenamed.dropna()

Ahora que ya se sabe que los datos de entrada tienen el formato adecuado se puede pasar a la división

In [6]:
ficheroDataFrameSplit = ficheroDataFrameRenamed.select(split(col("datos")," - "))
ficheroDataFrameSplit = ficheroDataFrameSplit.withColumnRenamed("split(datos,  - , -1)","datos")

ficheroDataFrameSplit.printSchema()


root
 |-- datos: array (nullable = true)
 |    |-- element: string (containsNull = true)



Se construye el dataframe 

In [7]:
dataFrameDatos= ficheroDataFrameSplit.select(col("datos").getItem(0),col("datos").getItem(1),col("datos").getItem(2),col("datos").getItem(3))
# Se renombran las columnas
dataFrameDatos = dataFrameDatos.withColumnRenamed("datos[0]","Fecha")
dataFrameDatos = dataFrameDatos.withColumnRenamed("datos[1]","Log")
dataFrameDatos = dataFrameDatos.withColumnRenamed("datos[2]","Module")
dataFrameDatos = dataFrameDatos.withColumnRenamed("datos[3]","Info")
dataFrameDatos.show()


+-------------------+--------+------+--------------------+
|              Fecha|     Log|Module|                Info|
+-------------------+--------+------+--------------------+
|01-11-2022 13:24:13|    INFO|pandas|Aliquam amet adip...|
|01-11-2022 13:24:17| WARNING| spark|Consectetur velit...|
|01-11-2022 13:24:20| WARNING|pandas|Porro etincidunt ...|
|01-11-2022 13:24:22|   ERROR|pandas|Tempora modi quiq...|
|01-11-2022 13:24:24| WARNING| spark|Quiquia etincidun...|
|01-11-2022 13:24:25|    INFO| spark|Non est porro por...|
|01-11-2022 13:24:26|CRITICAL|python|Porro labore eius...|
|01-11-2022 13:24:28|CRITICAL| numpy|Est ut tempora se...|
|01-11-2022 13:24:29|    INFO| spark|Sit dolorem dolor...|
|01-11-2022 13:24:30|CRITICAL| spark|Etincidunt aliqua...|
|01-11-2022 13:24:31|   DEBUG|python|Aliquam sed porro...|
|01-11-2022 13:24:32|CRITICAL| numpy|Numquam dolor ips...|
|01-11-2022 13:24:33|CRITICAL|python|Amet neque est ip...|
|01-11-2022 13:24:34|   DEBUG|python|Eius ut tempora i..

Ahora se tiene el dataframe con los datos separados por columnas, se tiene que transformar la fecha a timestamp

In [8]:
dataFrameDatos = dataFrameDatos.withColumn("Fecha",to_timestamp(col("Fecha"),"dd-MM-yyyy HH:mm:ss"))
dataFrameDatos.show()
dataFrameDatos.printSchema()

+-------------------+--------+------+--------------------+
|              Fecha|     Log|Module|                Info|
+-------------------+--------+------+--------------------+
|2022-11-01 13:24:13|    INFO|pandas|Aliquam amet adip...|
|2022-11-01 13:24:17| WARNING| spark|Consectetur velit...|
|2022-11-01 13:24:20| WARNING|pandas|Porro etincidunt ...|
|2022-11-01 13:24:22|   ERROR|pandas|Tempora modi quiq...|
|2022-11-01 13:24:24| WARNING| spark|Quiquia etincidun...|
|2022-11-01 13:24:25|    INFO| spark|Non est porro por...|
|2022-11-01 13:24:26|CRITICAL|python|Porro labore eius...|
|2022-11-01 13:24:28|CRITICAL| numpy|Est ut tempora se...|
|2022-11-01 13:24:29|    INFO| spark|Sit dolorem dolor...|
|2022-11-01 13:24:30|CRITICAL| spark|Etincidunt aliqua...|
|2022-11-01 13:24:31|   DEBUG|python|Aliquam sed porro...|
|2022-11-01 13:24:32|CRITICAL| numpy|Numquam dolor ips...|
|2022-11-01 13:24:33|CRITICAL|python|Amet neque est ip...|
|2022-11-01 13:24:34|   DEBUG|python|Eius ut tempora i..

Por último va a ser necesario realilzar un mapeo entre el nivel de incidencia y in valor numérico previamente establecido, pero antes se va a comprobar que todas las filas tienen un string esperado

In [9]:
log_mapping = {
    'CRITICAL': 50,
    'ERROR': 40,
    'WARNING': 30,
    'INFO': 20,
    'DEBUG': 10, 
    'NOTSET': 0}
log_mapping_spark = create_map([lit(x) for x in chain(*log_mapping.items())])

lista_incidencias = ['CRITICAL','ERROR','WARNING','INFO','DEBUG','NOTSET']

dataFrameDatos = dataFrameDatos.filter(col("Log").isin(lista_incidencias))

dataFrameDatos = dataFrameDatos.withColumn("Map",log_mapping_spark[col("Log")])
dataFrameDatos.show()


+-------------------+--------+------+--------------------+---+
|              Fecha|     Log|Module|                Info|Map|
+-------------------+--------+------+--------------------+---+
|2022-11-01 13:24:13|    INFO|pandas|Aliquam amet adip...| 20|
|2022-11-01 13:24:17| WARNING| spark|Consectetur velit...| 30|
|2022-11-01 13:24:20| WARNING|pandas|Porro etincidunt ...| 30|
|2022-11-01 13:24:22|   ERROR|pandas|Tempora modi quiq...| 40|
|2022-11-01 13:24:24| WARNING| spark|Quiquia etincidun...| 30|
|2022-11-01 13:24:25|    INFO| spark|Non est porro por...| 20|
|2022-11-01 13:24:26|CRITICAL|python|Porro labore eius...| 50|
|2022-11-01 13:24:28|CRITICAL| numpy|Est ut tempora se...| 50|
|2022-11-01 13:24:29|    INFO| spark|Sit dolorem dolor...| 20|
|2022-11-01 13:24:30|CRITICAL| spark|Etincidunt aliqua...| 50|
|2022-11-01 13:24:31|   DEBUG|python|Aliquam sed porro...| 10|
|2022-11-01 13:24:32|CRITICAL| numpy|Numquam dolor ips...| 50|
|2022-11-01 13:24:33|CRITICAL|python|Amet neque est ip.

Se comprueba que las columnas tienen el formato que esperamos

In [10]:
dataFrameDatos.printSchema()

root
 |-- Fecha: timestamp (nullable = true)
 |-- Log: string (nullable = true)
 |-- Module: string (nullable = true)
 |-- Info: string (nullable = true)
 |-- Map: integer (nullable = true)



Como era de esperar no se ha encontrado ningún valor a null, por lo tanto se puede continuar con la práctica

### Filtrado por módulo
Con el dataframe preprocesado como se desea, se va a comenzar a extraer información.
En un inicio se van a extraer las filas que coincidan con un valor de módulo.

In [11]:
modulo = "spark"
dataFrameDatos = dataFrameDatos.filter(modulo== col("Module"))
dataFrameDatos.show()

+-------------------+--------+------+--------------------+---+
|              Fecha|     Log|Module|                Info|Map|
+-------------------+--------+------+--------------------+---+
|2022-11-01 13:24:17| WARNING| spark|Consectetur velit...| 30|
|2022-11-01 13:24:24| WARNING| spark|Quiquia etincidun...| 30|
|2022-11-01 13:24:25|    INFO| spark|Non est porro por...| 20|
|2022-11-01 13:24:29|    INFO| spark|Sit dolorem dolor...| 20|
|2022-11-01 13:24:30|CRITICAL| spark|Etincidunt aliqua...| 50|
|2022-11-01 13:24:35| WARNING| spark|Quiquia numquam s...| 30|
|2022-11-01 13:24:37|CRITICAL| spark|Voluptatem dolor ...| 50|
|2022-11-01 13:24:38|CRITICAL| spark| Dolor neque ut non.| 50|
|2022-11-01 13:24:51|   DEBUG| spark|Modi est modi non...| 10|
|2022-11-01 13:24:51| WARNING| spark|Magnam numquam ut...| 30|
|2022-11-01 13:24:51|   DEBUG| spark|Modi quiquia cons...| 10|
|2022-11-01 13:24:51|    INFO| spark|Neque adipisci ma...| 20|
|2022-11-01 13:24:51| WARNING| spark|Velit numquam neq.

### Filtrado por palabra en string
En este apartado se va filtrar si una palabra se encuentra en el campo de información

In [12]:
palabra = "Porro"
dataFrameDatos = dataFrameDatos.filter(col("Info").contains(palabra))
dataFrameDatos.show()

+-------------------+--------+------+--------------------+---+
|              Fecha|     Log|Module|                Info|Map|
+-------------------+--------+------+--------------------+---+
|2022-11-01 13:24:51|   ERROR| spark|Porro dolorem sit...| 40|
|2022-11-01 13:24:51|CRITICAL| spark|Porro eius consec...| 50|
|2022-11-01 13:24:51|   DEBUG| spark|Porro sed dolore ...| 10|
|2022-11-01 13:24:51|    INFO| spark|Porro tempora qui...| 20|
|2022-11-01 13:24:51|    INFO| spark|Porro dolorem dol...| 20|
|2022-11-01 13:24:51|   ERROR| spark|Porro consectetur...| 40|
|2022-11-01 13:24:51| WARNING| spark|Porro est neque d...| 30|
|2022-11-01 13:24:51|   ERROR| spark|Porro ut eius sed...| 40|
|2022-11-01 13:24:51|    INFO| spark|Porro est numquam...| 20|
|2022-11-01 13:24:51| WARNING| spark|Porro quaerat ame...| 30|
|2022-11-01 13:24:51| WARNING| spark|Porro ut tempora ...| 30|
|2022-11-01 13:24:51| WARNING| spark|Porro dolorem con...| 30|
+-------------------+--------+------+------------------

### Filtrado por input de palabra
Se van a mostrar las filas que tengan un nivel de incidencia igual o superior que el nivel solicitado. De esta forma si se introduce como nivel mínimo "Warning", se deberían de mostrar las filas con niveles "Warning", "Error" y "Critical"

In [13]:
nivel = "WARNING"

# Primero se obtiene el valor numérico para el nivel
valor = log_mapping[nivel]

dataFrameDatos = dataFrameDatos.filter(valor <= col("Map"))
dataFrameDatos.show()

+-------------------+--------+------+--------------------+---+
|              Fecha|     Log|Module|                Info|Map|
+-------------------+--------+------+--------------------+---+
|2022-11-01 13:24:51|   ERROR| spark|Porro dolorem sit...| 40|
|2022-11-01 13:24:51|CRITICAL| spark|Porro eius consec...| 50|
|2022-11-01 13:24:51|   ERROR| spark|Porro consectetur...| 40|
|2022-11-01 13:24:51| WARNING| spark|Porro est neque d...| 30|
|2022-11-01 13:24:51|   ERROR| spark|Porro ut eius sed...| 40|
|2022-11-01 13:24:51| WARNING| spark|Porro quaerat ame...| 30|
|2022-11-01 13:24:51| WARNING| spark|Porro ut tempora ...| 30|
|2022-11-01 13:24:51| WARNING| spark|Porro dolorem con...| 30|
+-------------------+--------+------+--------------------+---+



### Filtrado por fecha
Se filtra para obtener todas las entradas del log posteriores al valor de fecha establecido

In [14]:
after = datetime.strptime('11/01/22 13:24:41', '%m/%d/%y %H:%M:%S')

dataFrameDatos = dataFrameDatos.filter(col("Fecha") >(after)) 
dataFrameDatos.show()

+-------------------+--------+------+--------------------+---+
|              Fecha|     Log|Module|                Info|Map|
+-------------------+--------+------+--------------------+---+
|2022-11-01 13:24:51|   ERROR| spark|Porro dolorem sit...| 40|
|2022-11-01 13:24:51|CRITICAL| spark|Porro eius consec...| 50|
|2022-11-01 13:24:51|   ERROR| spark|Porro consectetur...| 40|
|2022-11-01 13:24:51| WARNING| spark|Porro est neque d...| 30|
|2022-11-01 13:24:51|   ERROR| spark|Porro ut eius sed...| 40|
|2022-11-01 13:24:51| WARNING| spark|Porro quaerat ame...| 30|
|2022-11-01 13:24:51| WARNING| spark|Porro ut tempora ...| 30|
|2022-11-01 13:24:51| WARNING| spark|Porro dolorem con...| 30|
+-------------------+--------+------+--------------------+---+



### Programación defensiva
En esta práctica se han identificado problemas que harían que la ejecución fallara, pero no se ha sabido implementar las soluciones. Sería interesante protegerse frente a fallos en el formato de entrada, como por ejemplo que el carácter de separación no sea "-", aunque el alumno no ha sido capaz de implementarlo.
También interesaría protegerse frente a fechas incorrectas, que estuvieran en el pasado o que no cumplieran con el formato establecido, pero una vez más el alumno no ha sabido implementarlo.
Únicamente se ha protegido el comienzo del código para identificar las filas que pudieran estar vacías.
Por último se ha conseguido implementar la comprobación de que el nivel de incidencia se encuentra en la lista de valores conocidos